instead of all this, why not use a multimodal model and give it "tasks with specific prompts and inputs". Would be rather interesting for the orchestrator I guess right?

In [2]:
from typing import Any, Dict, List

import torch  # type: ignore
from fastapi import FastAPI, File, UploadFile, Form  # type: ignore
from fastapi.responses import JSONResponse  # type: ignore
import logging
import os
from dotenv import load_dotenv, find_dotenv  # type: ignore
from pydantic import BaseModel


# from app.general_inference_tools.modules import get_engine

torch.set_grad_enabled(False)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv(find_dotenv())

# app = FastAPI()
# BACKEND = os.getenv("BACKEND_URL", "http://localhost:8004")

from transformers import pipeline

In [3]:
TASK_MODEL_MAP = {
        "visual-question-answering" : "openbmb/MiniCPM-V-2",
        "document-question-answering": "naver-clova-ix/donut-base-finetuned-docvqa"
        }

# pipeline(1, model =2, trust_remote_code=)

class HFModelTaskConnector(BaseModel):
    """
    hf_task_tag: str - Actual task type on HF (eg: visual-question-answering
    subtask : str - Sub task (multiple subtasks can use the same model)
    model: str - Model name
    input_types: List[str] - Types of accepted inputs eg : *.jpeg, *.png
    """
    hf_task_tag: str
    subtask : str
    model: str
    input_types: List[str]

In [7]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

In [10]:
model = AutoModel.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True, torch_dtype=torch.bfloat16)
# For Nvidia GPUs support BF16 (like A100, H100, RTX3090)
# model = model.to(device='cuda', dtype=torch.bfloat16)
# For Nvidia GPUs do NOT support BF16 (like V100, T4, RTX2080)
#model = model.to(device='cuda', dtype=torch.float16)
# For Mac with MPS (Apple silicon or AMD GPUs).
# Run with `PYTORCH_ENABLE_MPS_FALLBACK=1 python test.py`
model = model.to(device='mps', dtype=torch.float16)

tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-V-2', trust_remote_code=True)
model.eval().cpu()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MiniCPMV(
  (llm): MiniCPMForCausalLM(
    (model): MiniCPMModel(
      (embed_tokens): Embedding(122753, 2304)
      (layers): ModuleList(
        (0-39): 40 x MiniCPMDecoderLayer(
          (self_attn): MiniCPMSdpaAttention(
            (q_proj): Linear(in_features=2304, out_features=2304, bias=False)
            (k_proj): Linear(in_features=2304, out_features=2304, bias=False)
            (v_proj): Linear(in_features=2304, out_features=2304, bias=False)
            (o_proj): Linear(in_features=2304, out_features=2304, bias=False)
            (rotary_emb): MiniCPMRotaryEmbedding()
          )
          (mlp): MiniCPMMLP(
            (gate_proj): Linear(in_features=2304, out_features=5760, bias=False)
            (up_proj): Linear(in_features=2304, out_features=5760, bias=False)
            (down_proj): Linear(in_features=5760, out_features=2304, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): MiniCPMRMSNorm()
          (post_attention_layernorm): Min

In [13]:
image = Image.open('../../sample_data/test_pdf.png').convert('RGB')
question = 'What is in the image?'
msgs = [{'role': 'user', 'content': question}]

res, context, _ = model.chat(
    image=image,
    msgs=msgs,
    context=None,
    tokenizer=tokenizer,
    sampling=True,
    temperature=0.7,
    use_cache = False
)

AttributeError: 'DynamicCache' object has no attribute 'get_max_length'